<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Decisions with Semantic Functions 

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks with their descriptions 
* Listing types of decision-making frameworks dynamically based on the type  

Semantic Kernel Functions are the core building blocks of functionality for intelligent AI orchestration. Semantic Kernel functions usually have a single responsibility to perform. For example, a Sementic Kernel function can: send an e-mail, call an API, recommend a reasoning framework for a high-stakes decision etc. 

What makes Semantic Kernel functions so special? Can't one just produce similar outcomes by using a prompt in an LLM or just writing a software program? In order to answer that, one has to look at what makes GenAI so innovative. The GenAI models have a unique ability to process instructions with reasoning and logic. This allows these models to behave almost like a human decision maker. Even with basic prompts, GenAI models perform reasonably well. However, providing GenAI models with additional functions allows them to gain access to business processes, data and basically anything a GenAI model should consider when performing AI orchestration. 

For example, imagine you want to prepare a great Thanksgiving dinner and want to get help from a GenAI cooking application to create a new recipe. You enter what you want todo and it comes up with the most delicious looking feast for Thanksgiving. However, there is a problem it used ingredients and recommended using cooking appliances that you do not own! You could enter all of the ingredients and keep prompting until it narrowed down the what you could realistically make. Wouldn't it be better if the GenAI cooking application had access to: your available ingredients, your time availability, kitchen appliances and even your allergic preferences. Now the GenAI model can craft a Thanksgiving feast understanding the parameters and data without having to be guided. This is exactly what Semantic Kernel functions provide.

Semantic Kernel functions come in two flavors: 
* Semantic functions
* Native functions

Semantic functions are basically **prompt instructions** that help guide GenAI LLMs around the specific guidelines it should adhere to when building the AI orchestration. Therefore, semantic functions are likely the first place most AI architects/engineers start when composing AI orchestration with Semantic Kernel. Semantic functions can also take in parameters for dynamic creaation of instructions.  

Below is an example of semantic function that can be used for facilitating decision-making. Note the **{{$decisionToMake}}** input where the parameter can be dynamically passed in to provide additional specificity.

```javascript
[DECISION FRAMEWORKS TO USE]
Price's Law
Pareto Principle
Laplace Rule of Succession
Eisenhower Matrix
Median Rule of Five
Second Order Thinking

[DECISION GUIDANCE]
Try to use the best fitting framework.  
Prefer using quantitative decision frameworks rather qualitative ones.  
Use the code interpreter to validate quantitative reasoning.

Use the decision frameworks above to help the user make the following decision:
{{$decisionToMake}}
```  

The image below illustrates the core value of semantic functions. 

<img width ="500px" src="https://learn.microsoft.com/en-us/semantic-kernel/media/semantic-function-explainer.png">

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [6]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.25.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.25.0-alpha"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.25.0 Microsoft.SemanticKernel.Plugins.Core, 1.25.0-alpha

In [7]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


### Step 2 - Create a Semantic Kernel - Semantic Function 

> "Decision frameworks are like maps. Use them to navigate complex decision-making terrain.  
>
> -- <cite>Unknown</cite> 

Semantic functions are constructs inside Semantic Kernel that are basically just prompt instructions. What makes the special is that they can be exposed as functions, thus it makes re-usability easy. Furthermore, future modules will show how they can faciliate composability into plugins. 

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with
var decisionPromptTemplate = """
Identify and list various decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.
""";

// Takes a regular prompt and creats a function that can be used to invoke the model
var semanticDecisionFunction = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate);

await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunction))
{
   Console.Write(streamChunk);
}

Sure! Here are several decision-making frameworks along with brief descriptions of how each supports better analysis and reasoning in various scenarios:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats):**
   - *Description:* SWOT analysis helps identify internal strengths and weaknesses, as well as external opportunities and threats faced by an organization or individual.
   - *Support:* By systematically evaluating positive and negative factors, this framework guides strategic planning and prioritizes actions based on situational analysis.

2. **Decision Matrix (Weighted Decision Matrix):**
   - *Description:* This involves listing options and criteria, then scoring each option against each criterion. Scores are weighted based on the importance of each criterion.
   - *Support:* It quantifies the decision-making process, allowing for a more objective comparison of options based on multiple factors.

3. **Cost-Benefit Analysis (CBA):**
   - *Description:* CBA involve

### Step 3 - Semantic Function Dynamic Decision Intelligence  

> "If the only tool you have is a hammer, you tend to see every problem as a nail."
>
> -- <cite>Abraham Maslow (Renowned American psychologist)</cite> 

Semantic functions can be dynamically composed using parameter placeholders, which can then be dynamically rendered using Kernel arguments. This allows the ease of passing in parameters and execution settings into the semantic functions.

Execute the cell below to view how the prompt can dynamically instruct the LLM to retrieve different types of decision-making frameworks. 

In [4]:
// Takes a prompt with input variables and creates a function that can be used to invoke the model
var decisionPromptTemplate = """
Identify and list {{$numberOfFrameworks}} decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in {{$decisionType}} decision-making scenarios.
""";

// Create the OpenAI prompt execution settings, try changing the settings to see how the output changes
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var semanticDecisionFunctionWithInput = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate, openAIPromptExecutionSettings);

// Dynamically set the number of frameworks and decision type -> strategic
var kernelArguments = new KernelArguments();
kernelArguments.Add("numberOfFrameworks", 3);
kernelArguments.Add("decisionType", "Long-Term strategic with probabilistic outcomes");

// Stream the output of the model
await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunctionWithInput, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Here are three decision-making frameworks that can enhance the quality of decisions, particularly in long-term strategic scenarios with probabilistic outcomes:

1. **Decision Trees:**
   - **Description:** Decision trees are graphical representations of possible solutions to a decision based on different conditions. They map out each possible outcome of a decision and the subsequent decisions that could follow.
   - **Support for Better Analysis and Reasoning:** Decision trees help in visualizing the various paths and outcomes of a decision, including the probabilities and potential payoffs of each path. This structured approach allows decision-makers to systematically evaluate the potential impacts of their choices, consider the likelihood of different scenarios, and make more informed decisions by comparing the expected values of different strategies.

2. **Scenario Planning:**
   - **Description:** Scenario planning involves creating detailed and plausible views of how th

In the example below, the number of frameworks to return has been changed as well as the type has been changed to **Quick to Implement for rapid Dicision-Making"**.  

In [10]:
// Dynamically set the number of frameworks and decision type -> requring fast action
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
kernelArguments.Add("numberOfFrameworks", 5);
kernelArguments.Add("decisionType", "Quick to Implement for rapid Dicision-Making");

// Stream the output of the model
await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunctionWithInput, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Here are five decision-making frameworks that can enhance the quality of decisions, particularly in scenarios requiring rapid decision-making:

1. **OODA Loop (Observe, Orient, Decide, Act):**
   - **Description:** The OODA Loop is a cyclical process that involves four stages: Observing the current situation, Orienting by analyzing and synthesizing information, Deciding on a course of action, and Acting on that decision.
   - **Support for Rapid Decision-Making:** This framework is designed for quick adaptation and response. It helps decision-makers rapidly process information, adjust to new data, and take swift, informed actions. It’s particularly useful in dynamic environments where conditions change rapidly.

2. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats):**
   - **Description:** SWOT Analysis involves identifying internal strengths and weaknesses, as well as external opportunities and threats related to a decision or situation.
   - **Support for Rapi